# Get audio features and genres from Spotify - V2

The idea, this time, is to find way to save the json response straight to a panda dataframe and later do all the joins

### Setup spotify

In [ ]:
# !pip install spotipy
# import needed libraries
import time
import numpy as np
import json
import io
import spotipy
import config
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

# connect to spotify
client_id = config.client_id # change with your keys
client_secret = config.client_secret # change with your keys
client_credential = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credential)

In [ ]:
df = pd.read_csv("spotify_it_2018_november_december.csv")

In [ ]:
df.head()

In [ ]:
# remove empty rows
df.dropna(inplace=True)
# convert string to numbers
df.Position = pd.to_numeric(df.Position, errors='coerce')
df.Streams = pd.to_numeric(df.Streams, errors='coerce')
df['2018-11-01 00:00:00'] = pd.to_datetime(df['2018-11-01 00:00:00'], errors='coerce')
# fill empty
df.Streams = df.Streams.fillna(0)
df.Position = df.Position.fillna(0)

## Preparing the dataset and getting the audio features per each track

About getting the audio features: https://developer.spotify.com/web-api/get-audio-features/

We will look first for the unique songs and loop trought their IDs to get the audio features.
(Hint: on 84000 rows I have 2014 unique songs... Looping for each song for each day it would have take me days)

In [ ]:
# number of unique tracks
df_track = df['URL'].drop_duplicates().reset_index()

# create an empty dataframe to store the data
df_track_data = pd.DataFrame()

from pandas.io.json import json_normalize

for i, row in df_track.iterrows():
    audio_ft = sp.audio_features(df_track['URL'][i])
    # song_data = sp.track(df_track['ID'][i])['artists'][0]['id'] - cant remember what
    # turns the list badly shape as a dictionary and normalize it as dataframe, later append it
    audio_data = json_normalize(audio_ft[0])
    df_track_data = df_track_data.append(audio_data)
    time.sleep(0.5)

#output the first row
df_track_data.head(1)

In [ ]:
# salvataggio di backup
df_track_data.to_csv("nov_dec_audio_features.csv", index=False)

## Merge and clean and save

At this point, I should merge back the data with the original chart and save it somewhere for further analysis

In [ ]:
#df_track_data.rename(columns={'id': 'ID'}, inplace=True)
df.rename(columns={'URL':'id'}, inplace=True)

In [ ]:
#join the files

df_song_data = df.merge(df_track_data, how='inner', on=['id'])
df_song_data.head(5)

In [ ]:
df_song_data.to_csv("new-song-data.csv", index=False)

In [ ]:
df_song_data.info()